# Human Connectome Project (HCP) Dataset loader

The HCP dataset comprises resting-state and task-based fMRI from a large sample of human subjects. The NMA-curated dataset includes time series data that has been preprocessed and spatially-downsampled by aggregating within 360 regions of interest.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, combinations_with_replacement
from tqdm import tqdm, trange
import rcca

In [2]:
#@title Figure settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

# Basic parameters

In [3]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "../group_project/hcp"
if not os.path.isdir(HCP_DIR):
  os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasesr parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in sec

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated multiple times in each subject
N_RUNS_REST = 4
N_RUNS_TASK = 2

# Time series data are organized by experiment, with each experiment
# having an LR and RL (phase-encode direction) acquistion
BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL",
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR",
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]

# You may want to limit the subjects used during code development.
# This will use all subjects:
subjects = range(N_SUBJECTS)

# for code development, we will first start using just one person's data
# subjects = range(1)

## Loading region information

Downloading either dataset will create the `regions.npy` file, which contains the region name and network assignment for each parcel.

Detailed information about the name used for each region is provided [in the Supplement](https://static-content.springer.com/esm/art%3A10.1038%2Fnature18933/MediaObjects/41586_2016_BFnature18933_MOESM330_ESM.pdf) to [Glasser et al. 2016](https://www.nature.com/articles/nature18933).

Information about the network parcellation is provided in [Ji et al, 2019](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6289683/).

In [4]:
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    myelin=regions[2].astype(np.float),
)

# Helper functions


## Data loading

In [5]:
def get_image_ids(name):
  """Get the 1-based image indices for runs in a given experiment.

    Args:
      name (str) : Name of experiment ("rest" or name of task) to load
    Returns:
      run_ids (list of int) : Numeric ID for experiment image files

  """
  run_ids = [
    i for i, code in enumerate(BOLD_NAMES, 1) if name.upper() in code
  ]
  if not run_ids:
    raise ValueError(f"Found no data for '{name}''")
  return run_ids

def load_timeseries(subject, name, runs=None, concat=True, remove_mean=True):
  """Load timeseries data for a single subject.
  
  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of experiment ("rest" or name of task) to load
    run (None or int or list of ints): 0-based run(s) of the task to load,
      or None to load all runs.
    concat (bool) : If True, concatenate multiple runs in time
    remove_mean (bool) : If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_tp array): Array of BOLD data values

  """
  # Get the list relative 0-based index of runs to use
  if runs is None:
    runs = range(N_RUNS_REST) if name == "rest" else range(N_RUNS_TASK)
  elif isinstance(runs, int):
    runs = [runs]

  # Get the first (1-based) run id for this experiment 
  offset = get_image_ids(name)[0]

  # Load each run's data
  bold_data = [
      load_single_timeseries(subject, offset + run, remove_mean) for run in runs
  ]

  # Optionally concatenate in time
  if concat:
    bold_data = np.concatenate(bold_data, axis=-1)

  return bold_data


def load_single_timeseries(subject, bold_run, remove_mean=True):
  """Load timeseries data for a single subject and single run.
  
  Args:
    subject (int): 0-based subject ID to load
    bold_run (int): 1-based run index, across all tasks
    remove_mean (bool): If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_path = f"{HCP_DIR}/subjects/{subject}/timeseries"
  bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts

def load_evs(subject, name, condition):
  """Load EV (explanatory variable) data for one task condition.

  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of task
    condition (str) : Name of condition

  Returns
    evs (list of dicts): A dictionary with the onset, duration, and amplitude
      of the condition for each run.

  """
  evs = []
  for id in get_image_ids(name):
    task_key = BOLD_NAMES[id - 1]
    ev_file = f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{condition}.txt"
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev = dict(zip(["onset", "duration", "amplitude"], ev_array))
    evs.append(ev)
  return evs

## Task-based analysis

In [6]:
def condition_frames(run_evs, skip=0):
  """Identify timepoints corresponding to a given condition in each run.

  Args:
    run_evs (list of dicts) : Onset and duration of the event, per run
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    frames_list (list of 1D arrays): Flat arrays of frame indices, per run

  """
  frames_list = []
  for ev in run_evs:

    # Determine when trial starts, rounded down
    start = np.floor(ev["onset"] / TR).astype(int)

    # Use trial duration to determine how many frames to include for trial
    duration = np.ceil(ev["duration"] / TR).astype(int)

    # Take the range of frames that correspond to this specific trial
    frames = [s + np.arange(skip, d) for s, d in zip(start, duration)]

    frames_list.append(np.concatenate(frames))

  return frames_list


def selective_average(timeseries_data, ev, skip=0):
  """Take the temporal mean across frames for a given condition.

  Args:
    timeseries_data (array or list of arrays): n_parcel x n_tp arrays
    ev (dict or list of dicts): Condition timing information
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    avg_data (1D array): Data averagted across selected image frames based
    on condition timing

  """
  # Ensure that we have lists of the same length
  if not isinstance(timeseries_data, list):
    timeseries_data = [timeseries_data]
  if not isinstance(ev, list):
    ev = [ev]
  if len(timeseries_data) != len(ev):
    raise ValueError("Length of `timeseries_data` and `ev` must match.")

  # Identify the indices of relevant frames
  frames = condition_frames(ev, skip)

  # Select the frames from each image
  selected_data = []
  for run_data, run_frames in zip(timeseries_data, frames):
    run_frames = run_frames[run_frames < run_data.shape[1]]
    selected_data.append(run_data[:, run_frames])

  # Take the average in each parcel
  avg_data = np.concatenate(selected_data, axis=-1).mean(axis=-1)

  return avg_data

# Task analyses

Description of each task, task timing, and conditions is located [here](https://protocols.humanconnectome.org/HCP/3T/task-fMRI-protocol-details.html).

These are the condition names for each task:

```
- MOTOR: cue, lf, lh, rf, rh, t
- WM:
    0bk_body, 0bk_faces, 0bk_nir, 0bk_placed, 0bk_tools, 
    2bk_body, 2bk_faces, 2bk_nir, 2bk_placed, 2bk_tools,
    0bk_cor, 0bk_err,
    2bk_cor, 2bk_err,
    all_bk_cor, all_bk_err
- EMOTION: fear, neut
- GAMBLING: loss, loss_event, win, win_event, neut_event
- LANGUAGE:
    cue,
    math, story
    present_math, present_story,
    question_math, question_story,
    response_math, response_story
- RELATIONAL: error, match, relation
- SOCIAL: mental_resp, mental, other_resp, rnd
```

## Cacluate Task-wise functional connectivity matrix

In [7]:
def selective_fc_mat(timeseries_data, ev, combs, skip=0, level="network"):
  """Take the temporal mean across frames for a given condition.

  Args:
    timeseries_data (array or list of arrays): n_parcel x n_tp arrays
    ev (dict or list of dicts): Condition timing information
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag
    level (str) : to which level will it reduce the data. either 'parcel' or 'network'

  Returns:
    avg_data (1D array): Data averagted across selected image frames based
    on condition timing

  """
  # Ensure that we have lists of the same length
  if not isinstance(timeseries_data, list):
    timeseries_data = [timeseries_data]
  if not isinstance(ev, list):
    ev = [ev]
  if len(timeseries_data) != len(ev):
    raise ValueError("Length of `timeseries_data` and `ev` must match.")
  if level not in ["network", "parcels"]:
    raise ValueError("level needs to be either `network` or `parcels`.")
  
  # Identify the indices of relevant frames
  frames = condition_frames(ev, skip)
  # Select the frames from each image
  selected_data = []
  # test_data = []
  for run_data, run_frames in zip(timeseries_data, frames):
    run_frames = run_frames[run_frames < run_data.shape[1]]
    selected_data.append(run_data[:, run_frames])
  # reduce the dimension
  # reduced_data = np.array(selected_data) # np.array(selected_data).reshape(360, len(frames) * len(run_frames)) 
  for i in range(len(selected_data)):
    if i == 0:
      tmp_data = np.array(selected_data[0])
    else:
      tmp_data = np.hstack((tmp_data, np.array(selected_data[i])))
  reduced_data = tmp_data

  # calculate the functional connectivity map
  fc_data = np.corrcoef(reduced_data)
  # connectivity between parcel 1 and parcel 4 would be fc_data[0,3] or fc_data[3,0]
    
  # calculate mean connectivity between different network  
  if level == "network":
    result = []
    for net1, net2 in combs:
      net1_parcels = [i for i,v in enumerate(network) if v == net1] # network: a list of lenght 360
      net2_parcels = [i for i,v in enumerate(network) if v == net2]
      result.append(np.mean(fc_data[np.ix_(net1_parcels,net2_parcels)]))
  elif level == "parcels":
    result = fc_data

  return result

In [8]:
task_cond = {
  "motor": ["lf", "rf", "lh", "rh", "t"],
  "wm": ["0bk_body", "0bk_faces", "0bk_places", "0bk_tools", 
         "2bk_body", "2bk_faces", "2bk_places", "2bk_tools"],
  "emotion": ["fear", "neut"],
  "gambling": ["win", "loss"],
  "language": ["story", "math"],
  "relational": ["relation", "match"],
  "social": ["mental", "rnd"]
}

In [9]:
network_fc_all = {}
network = region_info["network"]
network_names = np.unique(network)
combs = list(combinations_with_replacement(network_names, 2))

for subject in tqdm(subjects):
  # timeseries_all[subject] = {}
  for task, conditions in task_cond.items():
    if subject == 0:
      network_fc_all[task] = {}
    tmp_timeseries = load_timeseries(subject, task, concat=False)
    for cond in conditions:
      # initialize for first subject
      if subject == 0:
        network_fc_all[task][cond] = np.zeros((len(subjects), len(combs)))
      ev = load_evs(subject, task, cond)
      network_fc_all[task][cond][subject,:] = selective_fc_mat(tmp_timeseries, ev, combs, skip=0)

100%|██████████| 339/339 [01:54<00:00,  2.97it/s]


## Save data into pickle

In [10]:
import pickle
with open('network_summary.dat', 'wb') as f:
    pickle.dump(network_fc_all, f)

In [155]:
with open('network_summary.dat', 'rb') as f:
    new_dat = pickle.load(f)

In [156]:
new_dat.keys()

dict_keys(['motor', 'wm', 'emotion', 'gambling', 'language', 'relational', 'social'])

## Creating Resting-state Functional Connectivity Matrix

In [11]:
network = region_info["network"]
network_names = np.unique(network)
combs = list(combinations_with_replacement(network_names, 2))

rest_network = []

for subject in tqdm(subjects):
  ts_concat = load_timeseries(subject, "rest")
  fc_rest = np.corrcoef(ts_concat)
  result = []
  for net1, net2 in combs:
    net1_parcels = [i for i,v in enumerate(network) if v == net1]
    net2_parcels = [i for i,v in enumerate(network) if v == net2]
    result.append(np.mean(fc_rest[np.ix_(net1_parcels,net2_parcels)]))
  rest_network.append(result)

100%|██████████| 339/339 [00:09<00:00, 35.71it/s]


In [147]:
# convert rest_network to a numpy array
rest_network = np.array(rest_network)

## Applying CCA

In [164]:
from sklearn.cross_decomposition import CCA
cca = CCA(n_components=2)

In [165]:
# data preparation
X = rest_network
Y = network_fc_all['language']['math']
Y.shape

(339, 78)

In [167]:
cca.fit(X, Y)

CCA(copy=True, max_iter=500, n_components=2, scale=True, tol=1e-06)

In [168]:
# apply dimension reduction technique -> subject x n_components
X_r, Y_r = cca.transform(X, Y)

In [169]:
X_r.shape

(339, 2)

## Apply Pyrcca

In [177]:
import rcca

In [181]:
cca = rcca.CCA(kernelcca = False, reg = 0., numCC = 2)

In [182]:
X_train = rest_network[:len(subjects)//2]
Y_train = network_fc_all['language']['math'][:len(subjects)//2]
X_test  = rest_network[len(subjects)//2:]
Y_test  = network_fc_all['language']['math'][len(subjects)//2:]

In [183]:
cca.train([X_train, Y_train])
corrs = cca.validate([X_test, Y_test])
ev = cca.compute_ev([X_test, Y_test])

Training CCA, kernel = None, regularization = 0.0000, 2 components
Computing explained variance for component #1
Computing explained variance for component #2


In [212]:
# data preparation
X = rest_network # n_subject x connectivity measures (78 = 66 (12C2) + 12)
Y = np.hstack((network_fc_all['emotion']['neut'], network_fc_all['emotion']['fear']))
print(Y.shape)


from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=True, with_std=True)
X_sc = sc.fit_transform(X)
Y_sc = sc.fit_transform(Y)

(339, 156)


In [264]:
cca = rcca.CCA(kernelcca = False, reg = 1e-1, numCC = 5)
cca.train([X_sc, Y_sc])
print('Canonical Correlation Per Component Pair:',cca.cancorrs)
print('% Shared Variance:',cca.cancorrs**2)

Training CCA, kernel = None, regularization = 0.1000, 5 components
Canonical Correlation Per Component Pair: [0.95094805 0.94087171 0.93515026 0.93541459 0.9314675 ]
% Shared Variance: [0.9043022  0.88523957 0.87450601 0.87500045 0.8676317 ]


In [257]:
cca = rcca.CCACrossValidate(regs=[1e-1, 1, 1e2, 1e3, 1e4, 1e5], numCCs=[2, 3, 4, 5, 6, 7, 8, 9])
cca.train([X_sc, Y_sc])
print('Canonical Correlation Per Component Pair:',cca.cancorrs)
print('% Shared Variance:',cca.cancorrs**2)


Canonical Correlation Per Component Pair: [0.48340104 0.13304219 0.14352808 0.1332709  0.26647726 0.23268404
 0.22725339 0.1985611 ]
% Shared Variance: [0.23367656 0.01770022 0.02060031 0.01776113 0.07101013 0.05414186
 0.0516441  0.03942651]


In [255]:
print(f'Optimal number of components: {cca.best_numCC} \nOptimal regularization coefficient: {cca.best_reg:f}')

Optimal number of components: 8 
Optimal regularization coefficient: 100000.000000


## Permutation Test

First, prepare the FC map for the functional data, which takes up around 8 gigs of memory.

In [12]:
network_fc_parcels = {}

for subject in tqdm(subjects):
  for task, conditions in task_cond.items():
    # initialize for first subject
    if subject == 0:
      network_fc_parcels[task] = {}
    tmp_timeseries = load_timeseries(subject, task, concat=False)
    for cond in conditions:
      # initialize for first subject
      if subject == 0:
        network_fc_parcels[task][cond] = []
      ev = load_evs(subject, task, cond)
      network_fc_parcels[task][cond].append(selective_fc_mat(tmp_timeseries, ev, combs, skip=0, level = "parcels"))

100%|██████████| 339/339 [00:16<00:00, 20.50it/s]


then, prepare FC map for the rest data.

In [13]:
network_rest_parcels = []

for subject in tqdm(subjects):
  ts_concat = load_timeseries(subject, "rest")
  fc_rest = np.corrcoef(ts_concat)
  network_rest_parcels.append(fc_rest)

100%|██████████| 339/339 [00:05<00:00, 65.13it/s]


Now, preprocess the data and perform CCA every iteration.

In [16]:
permut_iter = 5000
network = region_info["network"]
network_names = np.unique(network)
combs = list(combinations_with_replacement(network_names, 2))
designate_task = "language"

# create pre_defined random network set
random_networks = []
for _ in range(permut_iter):
  np.random.shuffle(network)
  random_networks.append(network.copy())
# to save results
permut_rsq = []

In [ ]:
for i in trange(permut_iter):
  
  ##########################################
  ############# Preprocessing ##############
  ##########################################
  # randomly shuffle network every iteration
  tmp_network = random_networks[i]
  
  # data preprocessing part per subject
  X_rest = np.zeros((len(subjects), len(combs)))
  Y_tmp = {}
  for subj in subjects:
    # preprocess the resting data
    for j, nets in enumerate(combs):
      net1_parcels = [i for i,v in enumerate(tmp_network) if v == nets[0]]
      net2_parcels = [i for i,v in enumerate(tmp_network) if v == nets[1]]
      X_rest[subj,j] = np.mean(network_rest_parcels[subj][np.ix_(net1_parcels,net2_parcels)])
        
      # preprocess the functional data
      for cond in task_cond[designate_task]:
        if subj == 0:
          Y_tmp[cond] = np.zeros((len(subjects), len(combs)))
        Y_tmp[cond][subj,j] = np.mean(network_fc_parcels[designate_task][cond][subj][np.ix_(net1_parcels,net2_parcels)])
  
  for i, cond in enumerate(Y_tmp.keys()):
    if i == 0:
      Y_task = Y_tmp[cond]
    else:
      Y_task = np.hstack((Y_task, Y_tmp[cond]))
      
  ##########################################
  #############      CCA      ##############
  ##########################################
  cca = rcca.CCA(kernelcca = False, reg = 0., numCC = 5, verbose=False)
  cca.train([X_rest, Y_task])
  permut_rsq.append(cca.cancorrs**2)

  2%|▏         | 108/5000 [10:00<7:27:48,  5.49s/it]

In [ ]:
final_data = np.array(permut_rsq)
np.save('permutation_shared_variances', final_data)